### Что изменено?
- Аналогичные лабе 2 изменения (чуть более слабая модель, [немного промпт-инженеринга](https://www.youtube.com/watch?v=Va4_6QtKAwQ))
- Регулируемое число примеров для few-shot промпта
- Для каждого ответа с категорией, которой нет в списке, делаем ещё один запрос модели, чтобы она попробовала уточнить категорию самостоятельно
- ~~Просить модель "подумать" перед каждым ответом, чтобы обосновать ход "мыслей", а лишь затем дать однословный ответ~~

In [1]:
!pip install datasets -q
!pip install transformers -q
!pip install accelerate -q
!pip install fuzzywuzzy -q
!pip install python-Levenshtein -q

# https://stackoverflow.com/questions/53247985/tqdm-4-28-1-in-jupyter-notebook-intprogress-not-found-please-update-jupyter-an
!pip install ipywidgets -q
!pip install jupyterlab -q

In [2]:
from typing import Dict, List, Union

import datasets
import transformers

from fuzzywuzzy import fuzz, process

import sklearn.metrics as m
from tqdm.notebook import tqdm
import random

### Функция для создания промптов к модели

**[Здесь и находится ключевое отличие few-shot подхода от zero-shot](https://courses.sberuniversity.ru/llm-gigachat/2/3/2)**

In [3]:
def prepare_message_for_llm(
        text: Union[str, List[str]],
        examples: Dict[str, List[str]],
        prompt_system: str,
        prompt_user: str
    ) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:

    assert len(examples) >= 2, f'Ожидалось 2+ категорий, получено {len(examples)}'

    categories = list(examples.keys())
    categories_as_string = '; '.join(categories)
    # print(examples, categories)

    prompt_user_full = f'{prompt_user}'

    for category in categories:
        if (len(examples[category]) > 0):
            for example in examples[category]:
                prompt_user_full += f'\nПРИМЕР: {example} ' \
                                    f'\nОТВЕТ: {category} '
            # prompt += f'Текст: {" ".join(examples[cur].split())}\nВаш ответ: {cur}\n'

    prompt_user_full += f'\nЗАДАНИЕ: {text} \nСПИСОК ТЕМ (выберите ОДНУ тему СТРОГО ' \
                        f'из этого списка): "{categories_as_string}" \nВАШ ОТВЕТ: '

    messages = [
        {
            'role': 'system',
            'content': prompt_system
        },
        {
            'role': 'user',
            'content': prompt_user_full
        }
    ]

    return {'message_for_llm': messages}

# Pipeline

### Загрузим модель

In [4]:
try:
    print(LLM_PIPELINE)
except:
    LLM_PIPELINE = transformers.pipeline(model='Qwen/Qwen2.5-3B-Instruct', device_map='cuda:0', torch_dtype='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Загрузим датасет

In [5]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = datasets.load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = datasets.load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = datasets.load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

### Выделим категории

In [6]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Категории классификации текстов: \n{list_of_categories}')

Категории классификации текстов: 
['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [7]:
print(validation_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})


### Выделим случайные примеры

Тут добавил фичу чтобы менять число примеров, начиная с 3 примеров она или почти не влияет или вообще ухудшает

In [8]:
amount_of_examples = 2
random.seed(2024)

examples_by_categories = dict()

for current_category in list_of_categories:
    category_examples = []

    for ex_no in range(amount_of_examples):
        category_examples.append(
            random.choice(
                train_set.filter(lambda it: it['category'] == current_category)['text']
            )
        )

    examples_by_categories[current_category] = category_examples
    print(f'Категория: "{current_category}" \n{examples_by_categories[current_category]}\n')

Категория: "entertainment" 
['Это когда люди посещают место, которое очень отличается от их обычной повседневной жизни, чтобы расслабиться и развлечься.', 'Вечер начал певец Санджу Шарма, за ним выступил Джай Шанкар Чаудхари. esented the chhappan bhog bhajan также. Ему аккомпанировал певец Раджу Кханделвал.']

Категория: "geography" 
['Пятнадцать из этих метеоритов связывают с метеоритным дождем, прошедшим в июле прошлого года.', 'Некогда древний город Смирна сегодня — современный, развитый и оживленный торговый центр, расположившийся вдоль огромного залива и окруженный горами.']

Категория: "health" 
['Тем, кто тренируется постоянно, требуется больше поддержки по причине негативного отношения к боли и для того, чтобы отличить хронические боли от чувства дискомфорта после обычных физических нагрузок.', 'Они научились мастерски делать ампутации, чтобы спасать пациентов от гангрены, и так же хорошо освоили жгут и артериальные зажимы для приостановки кровотока.']

Категория: "politics" 
[

### Обернём тексты в prompt для llm

In [9]:
prompt_system_1 = 'Вы — умный помощник, умеющий читать и анализировать тексты ' \
                  'на русском языке, всегда дающий продуманные верные ответы. '

In [10]:
prompt_user_1 = 'Прочтите, пожалуйста, следующий набор текстов ниже ' \
                'и определите, какая ОДНА тема из списка тем внизу ' \
                'НАИБОЛЕЕ представлена. В ответ напишите ОДНУ НАИБОЛЕЕ' \
                'подходящую тему из списка, больше ничего. Спасибо! '

In [11]:
validation_set_for_llm = validation_set.map(
    lambda it: prepare_message_for_llm(
        it['text'],
        examples_by_categories,
        prompt_system_1,
        prompt_user_1
    )
)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [12]:
print(validation_set_for_llm)

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})


In [13]:
print(validation_set_for_llm['message_for_llm'][0])

[{'content': 'Вы — умный помощник, умеющий читать и анализировать тексты на русском языке, всегда дающий продуманные верные ответы. ', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий набор текстов ниже и определите, какая ОДНА тема из списка тем внизу НАИБОЛЕЕ представлена. В ответ напишите ОДНУ НАИБОЛЕЕподходящую тему из списка, больше ничего. Спасибо! \nПРИМЕР: Это когда люди посещают место, которое очень отличается от их обычной повседневной жизни, чтобы расслабиться и развлечься. \nОТВЕТ: entertainment \nПРИМЕР: Вечер начал певец Санджу Шарма, за ним выступил Джай Шанкар Чаудхари. esented the chhappan bhog bhajan также. Ему аккомпанировал певец Раджу Кханделвал. \nОТВЕТ: entertainment \nПРИМЕР: Пятнадцать из этих метеоритов связывают с метеоритным дождем, прошедшим в июле прошлого года. \nОТВЕТ: geography \nПРИМЕР: Некогда древний город Смирна сегодня — современный, развитый и оживленный торговый центр, расположившийся вдоль огромного залива и окруженный горами. \nО

### Сгенерируем ответы

In [14]:
validation_pred = list(map(
    lambda x: LLM_PIPELINE(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))
validation_true = validation_set['category']

  0%|          | 0/99 [00:00<?, ?it/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [15]:
print(validation_pred[0])

[{'content': 'Вы — умный помощник, умеющий читать и анализировать тексты на русском языке, всегда дающий продуманные верные ответы. ', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий набор текстов ниже и определите, какая ОДНА тема из списка тем внизу НАИБОЛЕЕ представлена. В ответ напишите ОДНУ НАИБОЛЕЕподходящую тему из списка, больше ничего. Спасибо! \nПРИМЕР: Это когда люди посещают место, которое очень отличается от их обычной повседневной жизни, чтобы расслабиться и развлечься. \nОТВЕТ: entertainment \nПРИМЕР: Вечер начал певец Санджу Шарма, за ним выступил Джай Шанкар Чаудхари. esented the chhappan bhog bhajan также. Ему аккомпанировал певец Раджу Кханделвал. \nОТВЕТ: entertainment \nПРИМЕР: Пятнадцать из этих метеоритов связывают с метеоритным дождем, прошедшим в июле прошлого года. \nОТВЕТ: geography \nПРИМЕР: Некогда древний город Смирна сегодня — современный, развитый и оживленный торговый центр, расположившийся вдоль огромного залива и окруженный горами. \nО

In [16]:
print(m.classification_report(
    y_true=validation_true,
    y_pred=[x[-1]['content'] for x in validation_pred])
)

                    precision    recall  f1-score   support

     entertainment       0.83      0.56      0.67         9
         geography       0.67      0.50      0.57         8
            health       0.89      0.73      0.80        11
           history       0.00      0.00      0.00         0
          politics       1.00      0.71      0.83        14
science/technology       0.85      0.92      0.88        25
          security       0.00      0.00      0.00         0
            sports       1.00      0.75      0.86        12
            travel       0.60      0.90      0.72        20

          accuracy                           0.78        99
         macro avg       0.65      0.56      0.59        99
      weighted avg       0.83      0.78      0.79        99



### Находим некорректные ответы

In [17]:
validation_clarification_idx = []
validation_clarification_set = []

for i, conversation in enumerate(validation_pred):
    for entry in conversation:
        if entry['role'] == 'assistant' and entry['content'] not in list_of_categories:
            validation_clarification_idx.append(i)

In [ ]:
print(f'Ответы с выдуманными категориями ({len(validation_clarification_idx)}):')

for i in validation_clarification_idx:
    validation_clarification_entry = validation_set[i]
    validation_clarification_entry['text'] #+= f' (неверая тема: {validation_pred[i][-1]["content"]})'
    validation_clarification_set.append(validation_clarification_entry)
    
    print(f'№{int(i)} -- {validation_pred[i][-1]["content"]} ({validation_set[i]})')

Ответы с выдуманными категориями (2):
№42 -- history ({'index_id': 1450, 'category': 'politics', 'text': 'Эпоха Троецарствия была одним из самых кровавых периодов в истории древнего Китая. Тысячи людей погибли, сражаясь за место на троне в большом дворце в Сиане.'})
№47 -- security ({'index_id': 1883, 'category': 'politics', 'text': 'Досмотры на контрольно-пропускных пунктах также стали гораздо более пристальными после событий 11 сентября 2001 года.'})


In [19]:
validation_clarification_set = datasets.Dataset.from_list(validation_clarification_set)

print(validation_clarification_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 2
})


### Уточняем некорректные ответы

In [20]:
prompt_system_2 = 'Вы — умный ассистент, умеющий читать и анализировать тексты ' \
                  'на русском языке, всегда дающий продуманные верные ответы. '

In [21]:
prompt_user_2 = 'Робот выполнял разделение текстов на темы, однако ' \
                'не смог определить темы некоторых текстов однозначно. ' \
                'Ваша задача — выбрать одну тему из списка тем ниже. ' \
                'Если ваш ответ будет содержать тему, которой в списке ' \
                'ниже НЕТ, он будет считаться НЕКОРРЕКТНЫМ И ОТКЛОНЁН!'

In [22]:
validation_clarification_set_for_llm = validation_clarification_set.map(
    lambda it: prepare_message_for_llm(
        it['text'],
        examples_by_categories,
        prompt_system_2,
        prompt_user_2
    )
)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [23]:
print(validation_clarification_set_for_llm)

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 2
})


In [24]:
validation_clarification_pred = list(map(
    lambda x: LLM_PIPELINE(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_clarification_set_for_llm['message_for_llm'])
))
validation_clarification_true = validation_clarification_set['category']

  0%|          | 0/2 [00:00<?, ?it/s]

In [38]:
print(validation_clarification_pred[0])

[{'content': 'Вы — умный ассистент, умеющий читать и анализировать тексты на русском языке, всегда дающий продуманные верные ответы. ', 'role': 'system'}, {'content': 'Робот выполнял разделение текстов на темы, однако не смог определить темы некоторых текстов однозначно. Ваша задача — выбрать одну тему из списка тем ниже. Если ваш ответ будет содержать тему, которой в списке ниже НЕТ, он будет считаться НЕКОРРЕКТНЫМ И ОТКЛОНЁН!\nПРИМЕР: Это когда люди посещают место, которое очень отличается от их обычной повседневной жизни, чтобы расслабиться и развлечься. \nОТВЕТ: entertainment \nПРИМЕР: Вечер начал певец Санджу Шарма, за ним выступил Джай Шанкар Чаудхари. esented the chhappan bhog bhajan также. Ему аккомпанировал певец Раджу Кханделвал. \nОТВЕТ: entertainment \nПРИМЕР: Пятнадцать из этих метеоритов связывают с метеоритным дождем, прошедшим в июле прошлого года. \nОТВЕТ: geography \nПРИМЕР: Некогда древний город Смирна сегодня — современный, развитый и оживленный торговый центр, расп

### Объединяем результаты

In [26]:
combine_cnt = 0
for i in validation_clarification_idx:
    validation_pred[i] = validation_clarification_pred[combine_cnt]
    combine_cnt += 1

In [29]:
print(m.classification_report(
    y_true=validation_true,
    y_pred=[x[-1]['content'] for x in validation_pred])
)

                    precision    recall  f1-score   support

     entertainment       0.83      0.56      0.67         9
         geography       0.67      0.50      0.57         8
            health       0.89      0.73      0.80        11
          politics       1.00      0.79      0.88        14
science/technology       0.85      0.92      0.88        25
          security       0.00      0.00      0.00         0
            sports       1.00      0.75      0.86        12
            travel       0.60      0.90      0.72        20

          accuracy                           0.79        99
         macro avg       0.73      0.64      0.67        99
      weighted avg       0.83      0.79      0.79        99



### Постобработка текста

Делаем нечёткое сравнение строк по формуле Левенштейна, выбираем тему из списка с наименьшим расстоянием

In [34]:
validation_pred_norm = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    validation_pred
))

In [35]:
print(m.classification_report(
    y_true=validation_true,
    y_pred=validation_pred_norm)
)

                    precision    recall  f1-score   support

     entertainment       0.83      0.56      0.67         9
         geography       0.67      0.50      0.57         8
            health       0.89      0.73      0.80        11
          politics       1.00      0.79      0.88        14
science/technology       0.85      0.92      0.88        25
            sports       0.90      0.75      0.82        12
            travel       0.60      0.90      0.72        20

          accuracy                           0.79        99
         macro avg       0.82      0.73      0.76        99
      weighted avg       0.82      0.79      0.79        99



# Тестовые данные

In [39]:
test_set_for_llm = test_set.map(
    lambda it: prepare_message_for_llm(
        it['text'],
        examples_by_categories,
        prompt_system_1,
        prompt_user_1
    )
)

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [40]:
test_pred = list(map(
    lambda x: LLM_PIPELINE(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
test_true = test_set['category']

  0%|          | 0/204 [00:00<?, ?it/s]

In [41]:
print(m.classification_report(
    y_true=test_true,
    y_pred=[x[-1]['content'] for x in test_pred])
)

                            precision    recall  f1-score   support

                   culture       0.00      0.00      0.00         0
             entertainment       0.77      0.53      0.62        19
                 geography       0.92      0.71      0.80        17
                    health       0.82      0.82      0.82        22
            nature/animals       0.00      0.00      0.00         0
                  politics       0.93      0.90      0.92        30
        science/technology       0.90      0.88      0.89        51
                    sports       0.94      0.68      0.79        25
                technology       0.00      0.00      0.00         0
     transportation/travel       0.00      0.00      0.00         0
                    travel       0.66      0.88      0.75        40
weather/weather_conditions       0.00      0.00      0.00         0
        weather_and_nature       0.00      0.00      0.00         0

                  accuracy                    

In [42]:
test_clarification_idx = []
test_clarification_set = []

for i, conversation in enumerate(test_pred):
    for entry in conversation:
        if entry['role'] == 'assistant' and entry['content'] not in list_of_categories:
            test_clarification_idx.append(i)

In [ ]:
print(f'Ответы с выдуманными категориями ({len(test_clarification_idx)}):')

for i in test_clarification_idx:
    test_clarification_entry = test_set[i]
    test_clarification_entry['text'] #+= f' (неверая тема: {test_pred[i][-1]["content"]})'
    test_clarification_set.append(test_clarification_entry)
    
    print(f'№{int(i)} -- {test_pred[i][-1]["content"]} ({test_set[i]})')

test_clarification_set = datasets.Dataset.from_list(test_clarification_set)

Ответы с выдуманными категориями (6):
№28 -- technology ({'index_id': 61, 'category': 'science/technology', 'text': 'Во время своего двухчасового выступления он заявил: "Сегодня Apple собирается переосмыслить свой телефон. Мы войдём сегодня в историю."'})
№49 -- nature/animals ({'index_id': 1366, 'category': 'science/technology', 'text': 'Оцелоты любят поедать мелких животных. Если представится случай, они будут ловить обезьян, змей, грызунов и птиц. Почти все зверьки, на которых охотится оцелот, гораздо меньше него.'})
№75 -- weather/weather_conditions ({'index_id': 1801, 'category': 'travel', 'text': 'Во время метели достаточное количество снега для того, чтобы можно было застрять, может выпасть за очень короткий промежуток времени.'})
№80 -- transportation/travel ({'index_id': 855, 'category': 'travel', 'text': 'Вагоны MetroPlus и Metro имеются в каждом поезде. Вагоны первого вида всегда находятся в конце поезда, ближайшем к Кейптауну.'})
№175 -- culture ({'index_id': 414, 'category

In [46]:
test_clarification_set_for_llm = test_clarification_set.map(
    lambda it: prepare_message_for_llm(
        it['text'],
        examples_by_categories,
        prompt_system_2,
        prompt_user_2
    )
)

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [47]:
test_clarification_pred = list(map(
    lambda x: LLM_PIPELINE(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_clarification_set_for_llm['message_for_llm'])
))
test_clarification_true = test_clarification_set['category']

  0%|          | 0/14 [00:00<?, ?it/s]

In [48]:
combine_cnt = 0
for i in test_clarification_idx:
    test_pred[i] = test_clarification_pred[combine_cnt]
    combine_cnt += 1

In [49]:
print(m.classification_report(
    y_true=test_true,
    y_pred=[x[-1]['content'] for x in test_pred])
)

                    precision    recall  f1-score   support

     entertainment       0.77      0.53      0.62        19
         geography       0.92      0.71      0.80        17
            health       0.82      0.82      0.82        22
            nature       0.00      0.00      0.00         0
          politics       0.93      0.90      0.92        30
science/technology       0.89      0.92      0.90        51
            sports       0.94      0.68      0.79        25
            travel       0.65      0.88      0.74        40
           weather       0.00      0.00      0.00         0

          accuracy                           0.81       204
         macro avg       0.66      0.60      0.62       204
      weighted avg       0.84      0.81      0.82       204



In [50]:
test_pred_norm = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    test_pred
))

In [51]:
print(m.classification_report(
    y_true=test_true,
    y_pred=test_pred_norm)
)

                    precision    recall  f1-score   support

     entertainment       0.77      0.53      0.62        19
         geography       0.92      0.71      0.80        17
            health       0.78      0.82      0.80        22
          politics       0.93      0.90      0.92        30
science/technology       0.89      0.92      0.90        51
            sports       0.94      0.68      0.79        25
            travel       0.65      0.90      0.76        40

          accuracy                           0.82       204
         macro avg       0.84      0.78      0.80       204
      weighted avg       0.84      0.82      0.82       204



Код с этой же моделью, но без изменений логики, даёт f-score = 0.74.

~~Объявляю гойду!~~

In [ ]:
UNLOAD_MODEL = False

if UNLOAD_MODEL:
    try:
        del LLM_PIPELINE
    except:
        pass

    from torch.cuda import empty_cache
    empty_cache()
    from gc import collect
    collect()